[imapclient 사용법] https://bokyeong-kim.github.io/python/basic/2020/07/27/python-basic(5).html

In [1]:
import imapclient

In [2]:
EMAIL = 'mafal201121@gmail.com'  # 본인의 이메일 주소를 입력하세요.

# 암호를 입력합니다. (입력 시 암호가 표시되지 않습니다)
password = 'goyqgwyjnmprnigo'

# IMAP 서버에 연결하고 로그인합니다
imap_obj = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap_obj.login(EMAIL, password)


b'mafal201121@gmail.com authenticated (Success)'

In [3]:
# 'INBOX' 폴더를 선택합니다
imap_obj.select_folder('INBOX', readonly=True)


{b'PERMANENTFLAGS': (),
 b'FLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'$NotPhishing',
  b'$Phishing'),
 b'UIDVALIDITY': 1,
 b'EXISTS': 946,
 b'RECENT': 0,
 b'UIDNEXT': 1069,
 b'HIGHESTMODSEQ': 315962,
 b'READ-ONLY': [b'']}

In [48]:
# 최근 10개의 메일을 검색합니다
email_ids = imap_obj.search(['ALL'])[-10:]

In [49]:
email_ids

[1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063]

In [19]:
# messages = imap_obj.search(['FROM', 'mafal201116@gmail.com'])[-1:]
messages = imap_obj.search(['FROM', 'mafal2011@naver.com'])[-1:]

In [20]:
messages[-1]

1066

In [21]:
# 메일 본문을 가져옵니다
for email_id in messages:
    raw_message = imap_obj.fetch([email_id], ['BODY[]', 'FLAGS'])
    print(raw_message[email_id][b'BODY[]'].decode('utf-8'))



Delivered-To: mafal201121@gmail.com
Received: by 2002:ac4:cc91:0:b0:66e:db78:6003 with SMTP id z17csp2447981pin;
        Tue, 10 Oct 2023 00:12:37 -0700 (PDT)
X-Google-Smtp-Source: AGHT+IF+00QDDo0+6gPC4mAroeqpyCVCJS4NaFmXZNXeHt+xgTEDRWIX8Kp5/pXvg4q34icoQx19
X-Received: by 2002:a05:6a00:1255:b0:690:d718:8c6d with SMTP id u21-20020a056a00125500b00690d7188c6dmr17611648pfi.15.1696921957024;
        Tue, 10 Oct 2023 00:12:37 -0700 (PDT)
ARC-Seal: i=1; a=rsa-sha256; t=1696921957; cv=none;
        d=google.com; s=arc-20160816;
        b=NMVsyXDzFcGGt//BjRbhbi7XVvHSYeX+5QoYD/mRaC+wpBNmp1YDUT+QjHuEtOidYS
         quUwdBC428qsq9ch/cX82WnMSGuuCzmLgicdyy6bqXg+BgV6ZCUNqmN0Ygsgsj8au0jJ
         F9V8xLfu6+YwcZQBWot10takZM4wx+TYaczbeVZpJWCjublw3zyymmsq8Zrhz39ITX6J
         RyebOfdI4LIIL/TybkZnpMd5WtdSPheqsDqUszS4Qqo9dJ7VdKr2xUV/pCVCfLkRJNpE
         dcqPNiQcMvgTYwnyPSkZB2m+gGOjDcCagRf0vPmeks0EHGdmEsa3dozaIgTUiqHczavT
         CraA==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google

In [22]:
mail_Str = raw_message[email_id][b'BODY[]'].decode('utf-8')

In [7]:
from email.parser import Parser
from pathlib import Path
import email.policy

In [23]:
e = Parser(policy=email.policy.default).parsestr(mail_Str)

In [24]:
print(dict(e).keys())

dict_keys(['Delivered-To', 'Received', 'X-Google-Smtp-Source', 'X-Received', 'ARC-Seal', 'ARC-Message-Signature', 'ARC-Authentication-Results', 'Return-Path', 'Received-SPF', 'Authentication-Results', 'X-Originating-IP', 'X-Works-Smtp-Source', 'DKIM-Signature', 'X-Session-ID', 'MIME-Version', 'Message-ID', 'Date', 'From', 'Importance', 'To', 'Subject', 'X-Works-Send-Opt', 'Content-Type'])


In [25]:
dict(e)['Content-Type']

'multipart/alternative; boundary="-----Boundary-WM=_7f4c575de700.1696921955128"'

In [34]:
for i, body in enumerate(e.walk()):
    print(c_type := body["Content-Type"])
    if ("TEXT/" in c_type.upper() and
        "UTF-8" in c_type.upper()):
        print(body.get_content())

multipart/alternative; boundary="-----Boundary-WM=_7f4c575de700.1696921955128"
text/plain; charset="utf-8"
테스트발신

text/html; charset="utf-8"
<html><head><style>p{margin-top:0px;margin-bottom:0px;}</style></head><body><div style="font-size:14px; font-family:Gulim,굴림,sans-serif;">테스트발신</div></body></html><table style='display:none'><tr><td><img src="https://mail.naver.com/readReceipt/notify/?img=qZKZWzJ4M6drMxmsKxumK4tqKrF0Fx0opAvZKokvFzioFoM%2FFqbdK63SFzErtzFXp6UmKAb5W4d5W4pZMLlGWq%2FsW6J4M6mZKAuwKoJgpr9C16m5MreR.gif" border="0"/></td></tr></table>


In [41]:
imap_obj.logout()

b'LOGOUT Requested'

# 발신자 검색해서 가져오기

In [42]:
from imapclient import IMAPClient

# IMAP 서버에 연결
with IMAPClient(host='imap.gmail.com') as client:
    # 로그인 (이메일과 비밀번호로)
    client.login('your_email@gmail.com', 'your_password')
    
    # 'INBOX' 폴더를 선택
    client.select_folder('INBOX')
    
    # 메시지 검색: 이 경우, "example@example.com" 이메일 주소에서 온 메일을 검색
    messages = client.search(['FROM', 'mafal2011@naver.com'])
    
    # 메시지 내용 가져오기
    for msg_id, msg_data in client.fetch(messages, ['ENVELOPE']).items():
        envelope = msg_data[b'ENVELOPE']
        print('From: %s' % envelope.from_[0].mailbox.decode(), end='@')
        print(envelope.from_[0].host.decode())
        print('Subject: %s' % envelope.subject.decode())
        print('-' * 30)
    
    client.logout()


LoginError: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'